In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math

In [2]:
ps = pd.read_csv('patch_seq_log_mouse.csv')

In [149]:
#ps.head()

In [3]:
ps = ps[["Date", "User", "Post patch?", "Post patch pipette R", "Nucleus sucked in?", "Notes", "Internal version", "PCR cycles", "SM_QC_PF", "Resolution index", "Bad dates"]]
#ps.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
ps.dropna(subset = ["Post patch?"], inplace = True)
ps["User"] = ps["User"].astype("category")
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps = ps[ps["Bad dates"] != 'x']
ps = ps[ps["SM_QC_PF"] != "cnt"]
ps = ps[ps["PCR cycles"] == 21]
ps = ps.drop('Bad dates', 1)
ps["Date"] = pd.to_datetime(ps["Date"], format = '%y%m%d')
#ps["PCR cycles"].value_counts()
ps

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4968,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4969,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.923406
4970,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4971,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4972,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4973,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4974,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4975,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,NaN
4978,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4979,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000


#### defining counting functions

In [4]:
def get_count(x):    
    count = x["Post patch?"].count()
    return count

def calc_pct(part,total):
    pct = float(get_count(part)) / get_count(total)
    return pct

In [5]:
#total = ps["Post patch?"].count()
total = get_count(ps)
total
ps['Post patch?'].value_counts()

Nucleated             755
No-Seal               556
Partial-Nucleus       204
nucleus_present        93
Outside-Out            92
nucleus_absent         76
Entire-Cell            17
no_nucleus_visible      6
nucleus_visible         4
entire_cell             3
missing                 1
Outside-Out?            0
Nucleus                 0
outside-out             0
Outside-out             0
no -seal                0
no-seal                 0
nr                      0
nucelated               0
Entire cell             0
Name: Post patch?, dtype: int64

#### defining filtering functions

In [12]:
#input is a dataframe

def nucleated(x):
    nucleus = x[(x["Post patch?"] == "Nucleated") | (((x["Post patch?"] == "nucleus_visible") | (x["Post patch?"] == "nucleus_present")) & (x["Post patch pipette R"] >= 501))]
    return nucleus

def partial_nucleated(y):
    partial = y[(y["Post patch?"] == 'Partial-Nucleus') | (((y['Post patch?'] == 'nucleus_present') | (y['Post patch?'] == 'nucleus_visible')) & (y["Post patch pipette R"] <= 500))]
    return partial

def outside_out(z):
    outside = z[(z["Post patch?"] == 'Outside-Out') | (((z['Post patch?'] == 'nucleus_absent') | (z['Post patch?'] == 'no_nuclues_visible')) & (z['Post patch pipette R'] >= 501))]
    return outside

def no_seal(w): 
    no = w[(w["Post patch?"] == 'No-Seal') | (((w['Post patch?'] == 'nucleus_absent') | (w['Post patch?'] == 'no_nucleus_visible')) & (w['Post patch pipette R'] <= 500))]
    return no
    
def pass_only(df):
    pas = df[df["SM_QC_PF"] == "pass"]
    return pas

def gen_filter(df, col, cond):
    filtr = df[df[col] == cond]
    return filtr


# Post patch condition percentages

#### Filtering to nucleated patches only and calculating percentage

In [25]:
#nuc = ps[(ps["Post patch?"] == "Nucleated") | (ps["Post patch?"] == "Partial-Nucleus") | (ps["Post patch?"] == "nucleus_present") | (ps["Post patch?"] == "nucleus_visible")]
#nuc = ps[(ps["Post patch?"] == "Nucleated") | (((ps["Post patch?"] == "nucleus_visible") | (ps["Post patch?"] == "nucleus_present")) & (ps["Post patch pipette R"] >= 500))]
nuc = nucleated(ps)
nuc["Post patch pipette R"].value_counts()
#dic = {'Nucleated': 'test',
    #  'nucelated': 'Nucleated'
    # }

#ps['Post patch?'] = ps['Post patch?'].map(lambda x: dic[x])

1000.0    32
2000.0    15
3000.0     7
1200.0     4
1500.0     3
2500.0     3
1700.0     2
1100.0     2
1800.0     2
2300.0     1
980.0      1
1400.0     1
930.0      1
822.0      1
600.0      1
Name: Post patch pipette R, dtype: int64

In [13]:
#nuc_count = nuc["Post patch?"].count()
nuc_count = get_count(nuc)
nuc_count

831

In [14]:
#nuc_percent = float(nuc_count)/total
#nuc_percent
nuc_pct = calc_pct(nuc, ps) 
print "nucleated patch %:", nuc_pct

nucleated patch %: 0.459878251245


#### Filtering to partial-nucleated patches only and calculating percentage 

In [24]:
part_nuc = partial_nucleated(ps)
part_nuc["Post patch pipette R"].value_counts()

300.0    5
150.0    3
50.0     3
100.0    2
200.0    2
400.0    1
4.0      1
70.0     1
250.0    1
30.0     1
500.0    1
Name: Post patch pipette R, dtype: int64

In [16]:
part_nuc_pct = calc_pct(part_nuc, ps)
print "partial nucleated patch %:", part_nuc_pct

partial nucleated patch %: 0.124515771998


#### Filtering to outside-out patches only and calculating percentage

#### Filtering to no-seal patches only and calculating percentage

# Calculating nucleated patch percentages by user

In [159]:
P8_total = ps[ps["User"] == 'P8']
P8_total_count = P8_total['Post patch?'].count()
P8_total_count

447

In [160]:
#use post patch nucleated filter
P8_nuc = P8_total[P8_total['Post patch?'] == 'Nucleated']
P8_nuc_count = P8_nuc['Post patch?'].count()
P8_nuc_count

211

In [161]:
float(P8_nuc_count)/P8_total_count

0.4720357941834452

In [162]:
for riguser in ['P1','P2','P6','P8','P9','PA','PB']:
    user_total = ps[ps['User'] == riguser]
    user_total_count = user_total['Post patch?'].count()
    user_nuc = user_total[user_total['Post patch?'] == 'Nucleated']
    user_nuc_count = user_nuc['Post patch?'].count()
    user_percent = float(user_nuc_count)/(user_total_count)
    print riguser,': ', user_percent
    

P1 :  0.342490842491
P2 :  0.482142857143
P6 :  0.369863013699
P8 :  0.472035794183
P9 :  0.450549450549
PA :  0.28125
PB :  0.0888888888889


# Obtaining nucleus dependant on internal solution version

#### Creating dataframe for standard internal only

In [19]:
#standard_internal = ps[ps["Internal_version"] == "AiV2"]
std_int = gen_filter(ps, "Internal version", "AiV2")
std_int = std_int[(std_int["User"] == "P2") | (std_int["User"] == "P8")]
std_int["Internal version"].value_counts()
#standard_internal = standard_internal[(standard_internal["User"] == "P2") | (standard_internal["User"] == "P8")]
#standard_internal.head()
                       

AiV2    759
Name: Internal version, dtype: int64

In [10]:
#total_standard_internal = standard_internal["Post patch?"].count()
tot_std_int = get_count(std_int)
tot_std_int

759

#### Creating dataframe for alexa internal only

In [17]:
#alexa_internal = ps[ps["Internal version"] == "AiV2-50uM alexa 488"]
#alexa_internal = alexa_internal[(alexa_internal["User"] == "P2") | (alexa_internal["User"] == "P8")]
#alexa_internal
alexa_int = gen_filter(ps, "Internal version", "AiV2-50uM alexa 488")
alexa_int = alexa_int[(alexa_int["User"] == "P2") | (alexa_int["User"] == "P8")]
alexa_int["Internal version"].value_counts()

AiV2-50uM alexa 488    248
Name: Internal version, dtype: int64

In [20]:
#total_alexa_internal = alexa_internal["Post patch?"].count()
tot_alexa_int = get_count(alexa_int)
tot_alexa_int

248

#### Percent of nucleated patches with standard internal

In [21]:
#standard_internal_nuc = standard_internal[(standard_internal["Post patch?"] == "Nucleated") | (standard_internal["Post patch?"] == "Partial-Nucleus") | (standard_internal["Post patch?"] == "nucleus_present")|(standard_internal["Post patch?"] == "nucleus_visible")]
std_int_nuc = nucleated(std_int)
std_int_nuc
#standard_internal_nuc_count = standard_internal_nuc["Post patch?"].count()
std_int_nuc_pct = calc_pct(std_int_nuc, std_int)
std_int_nuc_pct
#print "standard internal:" , float(standard_internal_nuc_count)/total_standard_internal

0.5125164690382081

#### Percent of nucleated patched with alexa internal

In [168]:
#alexa_internal_nuc = alexa_internal[(alexa_internal["Post patch?"] == "Nucleated") | (alexa_internal["Post patch?"] == "Partial-Nucleus") | (alexa_internal["Post patch?"] == "nucleus_present")|(alexa_internal["Post patch?"] == "nucleus_visible")]
alexa_internal_nuc = nucleated(alexa_internal)
alexa_internal_nuc
#alexa_internal_nuc_count = alexa_internal_nuc["Post patch?"].count()
alexa_internal_nuc_pct = calc_pct(alexa_internal_nuc, alexa_internal)
alexa_internal_nuc_pct
#print "alexa internal:" , float(alexa_internal_nuc_count)/total_alexa_internal

0.532258064516129

# Rate of SMQC pass by internal solution verion

In [169]:
standard_internal.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
standard_internal_count = standard_internal["Post patch?"].count()
standard_internal_count

758

In [170]:
standard_internal_pass = standard_internal[standard_internal["SM_QC_PF"] == "pass"]
standard_internal_pass
#standard_internal_pass_count = standard_internal_pass["Post patch?"].count()
#standard_internal_pass_count                                    

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4969,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.923406
4971,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4972,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4973,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4974,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4975,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,NaN
5000,2017-04-20,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.719078
5001,2017-04-20,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
5002,2017-04-20,P2,Entire-Cell,NaN,NaN,NaN,AiV2,21.0,pass,0.719078
5003,2017-04-20,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.601033


In [171]:
print "Standard internal pass rate:" , float(standard_internal_pass_count)/standard_internal_count

Standard internal pass rate: 0.872031662269


In [172]:
alexa_internal.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
alexa_internal_count = alexa_internal["Post patch?"].count()
alexa_internal_count

188

In [173]:
alexa_internal_pass = alexa_internal[alexa_internal["SM_QC_PF"] == "pass"]
alexa_internal_pass_count = alexa_internal_pass["Post patch?"].count()
alexa_internal_pass_count

150

In [174]:
print "Alexa internal pass rate:" , float(alexa_internal_pass_count)/alexa_internal_count

Alexa internal pass rate: 0.797872340426


# Rate of SM_QC pass based on internal version and post patch condition

In [175]:
standard_nucleated_pass = pass_only(standard_internal_nuc)
standard_nucleated_pass_pct = calc_pct(standard_nucleated_pass,standard_internal_nuc)
standard_nucleated_pass_pct

0.9151670951156813

In [176]:
alexa_nucleated_pass = pass_only(alexa_internal_nuc)
alexa_nucleated_pass_pct = calc_pct(alexa_nucleated_pass,alexa_internal_nuc)
alexa_nucleated_pass_pct

0.8409090909090909

In [178]:
standard_internal_no_seal = no_seal(standard_internal)
standard_no_seal_pass = pass_only(standard_internal_no_seal)
standard_no_seal_pass_pct = calc_pct(standard_no_seal_pass,standard_internal_no_seal)
standard_no_seal_pass_pct

0.7975206611570248

In [179]:
alexa_internal_no_seal = no_seal(alexa_internal)
alexa_no_seal_pass = pass_only(alexa_internal_no_seal)
alexa_no_seal_pass_pct = calc_pct(alexa_no_seal_pass,alexa_internal_no_seal)
alexa_no_seal_pass_pct

0.4807692307692308